# ANOVA-Analyse: Abstimmungsverhalten nach Regionen

Dieses Notebook untersucht, wie stark das Abstimmungsverhalten in Schweizer Gemeinden von verschiedenen regionalen Faktoren abhängt:

1. **Röstigraben** (Sprachgebiete: DE, FR, IT, RM)
2. **Stadt-Land** (Städtisch, Intermediär, Ländlich)
3. **Grossregionen** (7 Regionen)

## Methodik: ANOVA und Effektstärke (η²)

Die **Analysis of Variance (ANOVA)** testet, ob sich die Mittelwerte zwischen Gruppen signifikant unterscheiden.

Die **Effektstärke η² (Eta-Quadrat)** gibt an, welcher Anteil der Gesamtvarianz durch die Gruppenzugehörigkeit erklärt wird:

| η² | Interpretation |
|----|-----------------|
| 0.01 | kleiner Effekt |
| 0.06 | mittlerer Effekt |
| 0.14+ | grosser Effekt |

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Style settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Color palettes
SPRACH_COLORS = {'Deutsch': '#E74C3C', 'Französisch': '#3498DB', 'Italienisch': '#2ECC71', 'Rätoromanisch': '#9B59B6'}
STADT_COLORS = {'Städtisch': '#E74C3C', 'Intermediär': '#F39C12', 'Ländlich': '#27AE60'}
REGION_PALETTE = 'Set2'

print("Libraries loaded.")

In [ ]:
# Load data
DB_PATH = '../data/processed/swiss_votings.db'
conn = sqlite3.connect(DB_PATH)

# Main query with features
query = """
SELECT
    v.municipality_id,
    v.municipality_name,
    v.voting_date,
    v.proposal_id,
    v.title_de,
    v.ja_prozent,
    mf.sprachgebiete,
    mf.staedtische_laendliche_gebiete,
    mf.grossregionen_der_schweiz
FROM v_voting_results_analysis v
INNER JOIN municipality_features mf ON v.municipality_id = mf.bfs_nr
WHERE v.ja_prozent IS NOT NULL
"""
df = pd.read_sql_query(query, conn)

# Load labels
labels_query = """
SELECT feature_name, code, label
FROM feature_labels
WHERE feature_name IN ('sprachgebiete', 'staedtische_laendliche_gebiete', 'grossregionen_der_schweiz')
"""
labels_df = pd.read_sql_query(labels_query, conn)
conn.close()

# Create readable labels
sprach_map = dict(zip(labels_df[labels_df['feature_name'] == 'sprachgebiete']['code'],
                      labels_df[labels_df['feature_name'] == 'sprachgebiete']['label']))
stadt_map = {1: 'Städtisch', 2: 'Intermediär', 3: 'Ländlich'}
region_map = {1: 'Région lémanique', 2: 'Espace Mittelland', 3: 'Nordwestschweiz',
              4: 'Zürich', 5: 'Ostschweiz', 6: 'Zentralschweiz', 7: 'Ticino'}

df['Sprachgebiet'] = df['sprachgebiete'].map(sprach_map)
df['Stadt_Land'] = df['staedtische_laendliche_gebiete'].map(stadt_map)
df['Grossregion'] = df['grossregionen_der_schweiz'].map(region_map)

print(f"Datensatz geladen: {len(df):,} Abstimmungsergebnisse")
print(f"Anzahl Abstimmungsvorlagen: {df['proposal_id'].nunique()}")
print(f"Anzahl Gemeinden: {df['municipality_id'].nunique()}")

In [ ]:
def calculate_anova(data, group_col, value_col='ja_prozent'):
    """Calculate ANOVA statistics including eta-squared"""
    groups = [group[value_col].values for name, group in data.groupby(group_col)]
    groups = [g for g in groups if len(g) > 0]
    
    if len(groups) < 2:
        return None
    
    # F-test
    f_stat, p_value = stats.f_oneway(*groups)
    
    # Eta-squared
    grand_mean = data[value_col].mean()
    ss_total = ((data[value_col] - grand_mean) ** 2).sum()
    ss_between = sum(len(g) * (g.mean() - grand_mean) ** 2 for g in groups)
    eta_sq = ss_between / ss_total if ss_total > 0 else 0
    
    return {'F': f_stat, 'p': p_value, 'eta_squared': eta_sq}

print("ANOVA function defined.")

---
## Übersicht: ANOVA-Ergebnisse aller 223 Abstimmungen

Laden der vorberechneten Ergebnisse:

In [ ]:
# Load pre-computed results
results_df = pd.read_csv('anova_results_full.csv')

# Summary by category
summary = results_df.groupby('category').agg({
    'eta_squared': ['mean', 'median', 'max'],
    'significant': ['sum', 'mean']
}).round(3)

summary.columns = ['Mittlere η²', 'Median η²', 'Max η²', 'Anzahl signifikant', 'Anteil signifikant']
summary['Anteil signifikant'] = (summary['Anteil signifikant'] * 100).round(1).astype(str) + '%'
summary

In [ ]:
# Distribution of effect sizes
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

categories = [
    ('Röstigraben (Sprachgebiete)', '#3498DB', 'Röstigraben'),
    ('Stadt-Land', '#E74C3C', 'Stadt-Land'),
    ('Grossregionen', '#2ECC71', 'Grossregionen')
]

for ax, (cat, color, title) in zip(axes, categories):
    data = results_df[results_df['category'] == cat]['eta_squared']
    
    ax.hist(data, bins=25, color=color, alpha=0.7, edgecolor='white')
    ax.axvline(data.mean(), color='black', linestyle='--', linewidth=2, label=f'Mittel: {data.mean():.3f}')
    ax.axvline(0.14, color='red', linestyle=':', linewidth=1.5, alpha=0.7, label='Grenze "gross"')
    
    ax.set_xlabel('Effektstärke (η²)')
    ax.set_ylabel('Anzahl Abstimmungen')
    ax.set_title(title)
    ax.legend(fontsize=9)
    ax.set_xlim(0, 0.85)

plt.suptitle('Verteilung der Effektstärken über alle 223 Abstimmungen', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('fig_effect_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

**Interpretation:**
- Der **Röstigraben** zeigt durchgehend starke Effekte (Mittel η² = 0.29)
- **Grossregionen** ähnlich stark (Mittel η² = 0.26)
- **Stadt-Land** zeigt kleinere, aber oft noch signifikante Effekte (Mittel η² = 0.04)

---
## 5 Beispiele mit besonders grossen Effekten

### Beispiel 1: Ernährungssouveränität (2018) - Röstigraben

**Volksinitiative «Für Ernährungssouveränität. Die Landwirtschaft betrifft uns alle»**

Die Initiative forderte eine Stärkung der einheimischen Landwirtschaft. Sie wurde mit 68.4% Nein abgelehnt, aber mit **extremen regionalen Unterschieden**.

In [ ]:
# Example 1: Ernährungssouveränität
proposal_id = 166
data = df[df['proposal_id'] == proposal_id].copy()
title = data['title_de'].iloc[0]
date = data['voting_date'].iloc[0]

# Calculate ANOVA
anova_result = calculate_anova(data, 'Sprachgebiet')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Boxplot
ax = axes[0]
order = ['Französisch', 'Italienisch', 'Deutsch', 'Rätoromanisch']
colors = [SPRACH_COLORS[s] for s in order]
box = sns.boxplot(data=data, x='Sprachgebiet', y='ja_prozent', order=order, 
                  palette=colors, ax=ax, width=0.6)
ax.set_ylabel('Ja-Anteil (%)', fontsize=12)
ax.set_xlabel('')
ax.set_ylim(0, 100)

# Add mean markers
means = data.groupby('Sprachgebiet')['ja_prozent'].mean()
for i, region in enumerate(order):
    ax.plot(i, means[region], 'D', color='white', markersize=8, markeredgecolor='black', markeredgewidth=1.5)
    ax.annotate(f'{means[region]:.1f}%', (i, means[region] + 4), ha='center', fontsize=10, fontweight='bold')

ax.set_title('Verteilung nach Sprachgebiet', fontsize=12)

# Violin plot for density
ax = axes[1]
sns.violinplot(data=data, x='Sprachgebiet', y='ja_prozent', order=order,
               palette=colors, ax=ax, inner='quartile')
ax.set_ylabel('Ja-Anteil (%)', fontsize=12)
ax.set_xlabel('')
ax.set_ylim(0, 100)
ax.set_title('Dichteverteilung (Violin Plot)', fontsize=12)

# Main title with stats
fig.suptitle(f'Ernährungssouveränität ({date})\n'
             f'F = {anova_result["F"]:.1f}, p < 0.001, η² = {anova_result["eta_squared"]:.3f} (sehr grosser Effekt)',
             fontsize=14, y=1.02)

plt.tight_layout()
plt.savefig('fig_example1_ernaehrung.png', dpi=150, bbox_inches='tight')
plt.show()

# Statistics table
stats_table = data.groupby('Sprachgebiet')['ja_prozent'].agg(['count', 'mean', 'std', 'min', 'max']).round(1)
stats_table.columns = ['N', 'Mittelwert', 'Std.Abw.', 'Min', 'Max']
stats_table = stats_table.loc[order]
print("\nDeskriptive Statistik:")
print(stats_table)

**Interpretation:** Die Westschweiz (52.8% Ja) stimmte deutlich anders ab als die Deutschschweiz (23.1% Ja) - eine Differenz von fast **30 Prozentpunkten**. η² = 0.78 bedeutet, dass 78% der Varianz im Abstimmungsverhalten durch die Sprachregion erklärt wird.

---
### Beispiel 2: Organspende-Gesetz (2022) - Röstigraben

**Änderung des Bundesgesetzes über die Transplantation von Organen, Geweben und Zellen (Widerspruchslösung)**

In [ ]:
# Example 2: Organspende
proposal_id = 200
data = df[df['proposal_id'] == proposal_id].copy()
title = data['title_de'].iloc[0]
date = data['voting_date'].iloc[0]

anova_result = calculate_anova(data, 'Sprachgebiet')

fig, ax = plt.subplots(figsize=(10, 6))

order = ['Französisch', 'Italienisch', 'Rätoromanisch', 'Deutsch']
colors = [SPRACH_COLORS[s] for s in order]

# Create boxplot with swarm overlay
sns.boxplot(data=data, x='Sprachgebiet', y='ja_prozent', order=order,
            palette=colors, ax=ax, width=0.5, fliersize=0)

# Add mean line
national_mean = data['ja_prozent'].mean()
ax.axhline(national_mean, color='gray', linestyle='--', linewidth=1.5, 
           label=f'Schweizer Mittel: {national_mean:.1f}%')
ax.axhline(50, color='red', linestyle=':', linewidth=1, alpha=0.5, label='50%-Linie')

# Add annotations
means = data.groupby('Sprachgebiet')['ja_prozent'].mean()
for i, region in enumerate(order):
    ax.annotate(f'{means[region]:.1f}%', (i, means[region] + 3), 
                ha='center', fontsize=11, fontweight='bold')

ax.set_ylabel('Ja-Anteil (%)', fontsize=12)
ax.set_xlabel('')
ax.set_ylim(0, 100)
ax.legend(loc='lower right')

ax.set_title(f'Organspende-Gesetz / Widerspruchslösung ({date})\n'
             f'F = {anova_result["F"]:.1f}, p < 0.001, η² = {anova_result["eta_squared"]:.3f}',
             fontsize=13)

plt.tight_layout()
plt.savefig('fig_example2_organspende.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nDifferenz FR vs DE: {means['Französisch'] - means['Deutsch']:.1f} Prozentpunkte")

**Interpretation:** Die Romandie befürwortete die Widerspruchslösung bei der Organspende mit 78%, während die Deutschschweiz knapp bei 51% lag. Ein klassisches Röstigraben-Thema.

---
### Beispiel 3: Massentierhaltungsinitiative (2022) - Stadt-Land

**Volksinitiative «Keine Massentierhaltung in der Schweiz»**

In [ ]:
# Example 3: Massentierhaltung (Stadt-Land)
proposal_id = 202
data = df[df['proposal_id'] == proposal_id].copy()
date = data['voting_date'].iloc[0]

anova_result = calculate_anova(data, 'Stadt_Land')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Boxplot
ax = axes[0]
order = ['Städtisch', 'Intermediär', 'Ländlich']
colors = [STADT_COLORS[s] for s in order]

sns.boxplot(data=data, x='Stadt_Land', y='ja_prozent', order=order,
            palette=colors, ax=ax, width=0.5)

means = data.groupby('Stadt_Land')['ja_prozent'].mean()
for i, region in enumerate(order):
    ax.annotate(f'{means[region]:.1f}%', (i, means[region] + 3), 
                ha='center', fontsize=11, fontweight='bold')

ax.axhline(50, color='red', linestyle=':', linewidth=1, alpha=0.5)
ax.set_ylabel('Ja-Anteil (%)', fontsize=12)
ax.set_xlabel('')
ax.set_ylim(0, 70)
ax.set_title('Boxplot nach Gemeindetyp', fontsize=12)

# Bar chart with error bars
ax = axes[1]
stats_data = data.groupby('Stadt_Land')['ja_prozent'].agg(['mean', 'std', 'count'])
stats_data = stats_data.loc[order]

# Calculate confidence intervals
stats_data['ci'] = 1.96 * stats_data['std'] / np.sqrt(stats_data['count'])

bars = ax.bar(order, stats_data['mean'], yerr=stats_data['ci'], 
              color=colors, alpha=0.8, capsize=8, edgecolor='black', linewidth=1)

for bar, (idx, row) in zip(bars, stats_data.iterrows()):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + row['ci'] + 1,
            f'{row["mean"]:.1f}%\n(n={int(row["count"])})', 
            ha='center', va='bottom', fontsize=10)

ax.set_ylabel('Ja-Anteil (%) mit 95%-KI', fontsize=12)
ax.set_ylim(0, 55)
ax.set_title('Mittelwerte mit Konfidenzintervall', fontsize=12)

fig.suptitle(f'Massentierhaltungsinitiative ({date})\n'
             f'F = {anova_result["F"]:.1f}, p < 0.001, η² = {anova_result["eta_squared"]:.3f}',
             fontsize=14, y=1.02)

plt.tight_layout()
plt.savefig('fig_example3_massentierhaltung.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nDifferenz Städtisch vs Ländlich: {means['Städtisch'] - means['Ländlich']:.1f} Prozentpunkte")

**Interpretation:** Städtische Gemeinden (36.1% Ja) stimmten deutlich tierfreundlicher ab als ländliche Gemeinden (25.4% Ja), wo die Landwirtschaft direkt betroffen ist. Der Stadt-Land-Graben zeigt sich hier klar.

---
### Beispiel 4: Liegenschaftssteuern auf Zweitliegenschaften (2025) - Grossregionen

**Bundesbeschluss über die kantonalen Liegenschaftssteuern auf Zweitliegenschaften**

In [ ]:
# Example 4: Liegenschaftssteuern (Grossregionen)
proposal_id = 222
data = df[df['proposal_id'] == proposal_id].copy()
date = data['voting_date'].iloc[0]

anova_result = calculate_anova(data, 'Grossregion')

fig, ax = plt.subplots(figsize=(12, 6))

# Sort by mean
means = data.groupby('Grossregion')['ja_prozent'].mean().sort_values(ascending=True)
order = means.index.tolist()

# Horizontal boxplot
sns.boxplot(data=data, y='Grossregion', x='ja_prozent', order=order,
            palette='Set2', ax=ax, orient='h')

# Add mean annotations
for i, region in enumerate(order):
    ax.annotate(f'{means[region]:.1f}%', (means[region] + 2, i), 
                va='center', fontsize=10, fontweight='bold')

ax.axvline(50, color='red', linestyle=':', linewidth=1.5, alpha=0.7, label='50%-Linie')
ax.set_xlabel('Ja-Anteil (%)', fontsize=12)
ax.set_ylabel('')
ax.set_xlim(0, 100)
ax.legend(loc='lower right')

ax.set_title(f'Liegenschaftssteuern auf Zweitliegenschaften ({date})\n'
             f'F = {anova_result["F"]:.1f}, p < 0.001, η² = {anova_result["eta_squared"]:.3f}',
             fontsize=13)

plt.tight_layout()
plt.savefig('fig_example4_liegenschaften.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nDifferenz Ostschweiz vs Région lémanique: {means['Ostschweiz'] - means['Région lémanique']:.1f} Prozentpunkte")

**Interpretation:** Die Ostschweiz und Zentralschweiz (>71% Ja) stimmten stark für die Besteuerung von Zweitliegenschaften, während die Région lémanique (40.6%) dagegen war. Die Tourismusregionen mit vielen Zweitwohnungsbesitzern aus der Romandie lehnten die Vorlage ab.

---
### Beispiel 5: CO2-Gesetz (2021) - Stadt-Land & Röstigraben kombiniert

**Bundesgesetz über die Verminderung von Treibhausgasemissionen**

In [ ]:
# Example 5: CO2-Gesetz (combined analysis)
proposal_id = 188
data = df[df['proposal_id'] == proposal_id].copy()
date = data['voting_date'].iloc[0]

# Two-way visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Stadt-Land
ax = axes[0]
anova_sl = calculate_anova(data, 'Stadt_Land')
order_sl = ['Städtisch', 'Intermediär', 'Ländlich']
sns.boxplot(data=data, x='Stadt_Land', y='ja_prozent', order=order_sl,
            palette=[STADT_COLORS[s] for s in order_sl], ax=ax)
means_sl = data.groupby('Stadt_Land')['ja_prozent'].mean()
for i, s in enumerate(order_sl):
    ax.annotate(f'{means_sl[s]:.1f}%', (i, means_sl[s] + 3), ha='center', fontweight='bold')
ax.set_ylabel('Ja-Anteil (%)')
ax.set_xlabel('')
ax.set_ylim(0, 80)
ax.set_title(f'Stadt-Land (η² = {anova_sl["eta_squared"]:.3f})')

# Sprachgebiet
ax = axes[1]
anova_sp = calculate_anova(data, 'Sprachgebiet')
order_sp = ['Französisch', 'Italienisch', 'Deutsch', 'Rätoromanisch']
sns.boxplot(data=data, x='Sprachgebiet', y='ja_prozent', order=order_sp,
            palette=[SPRACH_COLORS[s] for s in order_sp], ax=ax)
means_sp = data.groupby('Sprachgebiet')['ja_prozent'].mean()
for i, s in enumerate(order_sp):
    ax.annotate(f'{means_sp[s]:.1f}%', (i, means_sp[s] + 3), ha='center', fontweight='bold')
ax.set_ylabel('')
ax.set_xlabel('')
ax.set_ylim(0, 80)
ax.set_title(f'Sprachgebiet (η² = {anova_sp["eta_squared"]:.3f})')

# Interaction: Heatmap Stadt-Land x Sprachgebiet
ax = axes[2]
pivot = data.pivot_table(values='ja_prozent', index='Stadt_Land', columns='Sprachgebiet', aggfunc='mean')
pivot = pivot.loc[order_sl, ['Deutsch', 'Französisch', 'Italienisch']]
sns.heatmap(pivot, annot=True, fmt='.1f', cmap='RdYlGn', center=50, 
            ax=ax, cbar_kws={'label': 'Ja-Anteil (%)'})
ax.set_title('Interaktion Stadt-Land × Sprache')
ax.set_xlabel('')
ax.set_ylabel('')

fig.suptitle(f'CO2-Gesetz ({date})', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('fig_example5_co2.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nMittelwerte nach Region und Gemeindetyp:")
print(pivot.round(1))

**Interpretation:** Das CO2-Gesetz zeigt sowohl einen **Stadt-Land-Effekt** (Städte stimmten stärker Ja) als auch einen **Sprachregion-Effekt** (Romandie skeptischer). Die Heatmap zeigt, dass städtische Deutschschweizer Gemeinden am stärksten zustimmten, während ländliche Tessiner Gemeinden am skeptischsten waren.

---
## Zusammenfassung der 5 Beispiele

In [ ]:
# Summary table
examples = [
    {'Abstimmung': 'Ernährungssouveränität', 'Jahr': 2018, 'Kategorie': 'Röstigraben', 
     'η²': 0.780, 'Max Differenz': '30 PP (FR vs DE)', 'Interpretation': 'Sehr starker Röstigraben'},
    {'Abstimmung': 'Organspende-Gesetz', 'Jahr': 2022, 'Kategorie': 'Röstigraben',
     'η²': 0.779, 'Max Differenz': '28 PP (FR vs DE)', 'Interpretation': 'Sehr starker Röstigraben'},
    {'Abstimmung': 'Massentierhaltung', 'Jahr': 2022, 'Kategorie': 'Stadt-Land',
     'η²': 0.239, 'Max Differenz': '11 PP (Stadt vs Land)', 'Interpretation': 'Grosser Stadt-Land-Graben'},
    {'Abstimmung': 'Liegenschaftssteuern', 'Jahr': 2025, 'Kategorie': 'Grossregionen',
     'η²': 0.683, 'Max Differenz': '31 PP (Ost vs Léman)', 'Interpretation': 'Sehr starke regionale Differenz'},
    {'Abstimmung': 'CO2-Gesetz', 'Jahr': 2021, 'Kategorie': 'Kombiniert',
     'η²': 0.234, 'Max Differenz': '12 PP', 'Interpretation': 'Stadt-Land + Sprachregion'}
]

summary_examples = pd.DataFrame(examples)
print("Zusammenfassung der 5 Beispiele mit grossen Effekten:\n")
print(summary_examples.to_string(index=False))

---
## Fazit

Die ANOVA-Analyse zeigt:

1. **Der Röstigraben ist real und messbar**: Bei 99% aller Abstimmungen gibt es signifikante Unterschiede zwischen den Sprachregionen. Die mittlere Effektstärke (η² = 0.29) ist gross.

2. **Stadt-Land-Unterschiede sind kleiner, aber konsistent**: Bei 86% der Abstimmungen signifikant, aber mit kleinerer Effektstärke (η² = 0.04).

3. **Grossregionen zeigen starke Muster**: 100% signifikant mit hoher Effektstärke (η² = 0.26) - oft eine Kombination aus Sprach- und Stadt-Land-Effekten.

4. **Themenabhängigkeit**: Die stärksten Effekte zeigen sich bei:
   - Landwirtschaftsthemen (Ernährungssouveränität, Pestizide)
   - Gesundheitsthemen (Organspende, Krankenkassen)
   - Umweltthemen (CO2, Massentierhaltung)
   - Steuer- und Eigentumsthemen (Liegenschaftssteuern)